<a href="https://colab.research.google.com/github/Quicktorpedo/tp1_ml_ecc/blob/main/colabs/intro/Intro_to_Weights_%26_Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_&_Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{intro-colab} -->

<a href="https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_&_Biases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{intro-colab} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />
<!--- @wandbcode{intro-colab} -->

Use [W&B](https://wandb.ai/site?utm_source=intro_colab&utm_medium=code&utm_campaign=intro) for machine learning experiment tracking, model checkpointing, collaboration with your team and more. See the full W&B Documentation [here](https://docs.wandb.ai/).

In this notebook, you will create and track a machine learning experiment using a simple PyTorch model. By the end of the notebook, you will have an interactive project dashboard that you can share and customize with other members of your team. [View an example dashboard here](https://wandb.ai/wandb/wandb_example).

## Prerequisites

Install the W&B Python SDK and log in:

In [1]:
!pip install wandb -qU

In [2]:
# Log in to your W&B account
import wandb
import random
import math

In [3]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: josephogodja (josephogodja-ecole-centrale-casablanca) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# TP1 Machine Learning

## Importation des modules python adéquats

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (confusion_matrix, accuracy_score, precision_score,
                             recall_score, f1_score, roc_curve, auc, r2_score)
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier

### Code principal

In [ ]:
# 00 - Chargement du fichier CSV
df = pd.read_csv("/content/uci-secom.csv")

# 01 - Pré-traitement des données

# Suppression des colonnes avec plus de 5% de valeurs manquantes
missing_values = df.isnull().sum()
missing_percent = (missing_values / len(df)) * 100
missing_data = missing_values[missing_values > 0].sort_values(ascending=False)
missing_percent = missing_percent[missing_percent > 0].sort_values(ascending=False)

drop_cols = missing_percent[missing_percent > 5].index
df.drop(columns=drop_cols, inplace=True)

# Suppression des lignes avec au moins une valeur manquante
df.dropna(inplace=True)

# Suppression des doublons
df.drop_duplicates(inplace=True)

# Suppression des colonnes ayant des valeurs nulles ou identiques
cols_to_drop = [col for col in df.columns if df[col].nunique() <= 1]
df.drop(columns=cols_to_drop, inplace=True)

# Suppression de la colonne "Time" si elle existe
df.drop(columns=['Time'], errors='ignore', inplace=True)

# Encodage des variables catégoriques
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

# Retrait des colonnes avec une corrélation supérieure à 0.9
# Calcul la matrice de corrélation
corr_matrix = df.corr()

# Trouver les indices des colonnes avec une corrélation supérieure à 0.9
upper_triangle = corr_matrix.where(abs(corr_matrix) > 0.9).stack().index.tolist()

# Créer une liste de colonnes à supprimer
columns_to_drop = []
for col1, col2 in upper_triangle:
    if col1 != col2:  # Si ce ne sont pas la même colonne
        columns_to_drop.append(col2)

# Supprimer les colonnes avec une corrélation supérieure à 0.9
df_cleaned = df.drop(columns=columns_to_drop)

# Retrait des valeurs aberrantes au delà de l'intervalle [-1250;1250]
# Définition les seuils pour les valeurs aberrantes
upper_limit = 1250
lower_limit = -1250

# Identifier les colonnes contenant des valeurs aberrantes
columns_to_drop = []
for col in df_cleaned.columns:
    if (df_cleaned[col] > upper_limit).any() or (df_cleaned[col] < lower_limit).any():
        columns_to_drop.append(col)

# Supprimer les colonnes contenant des valeurs aberrantes
df_cleaned_no_outliers = df_cleaned.drop(columns=columns_to_drop)

# Création des vecteurs X (features) et Y (target)
X = df_cleaned_no_outliers.drop(columns=['Pass/Fail'])
Y = df_cleaned_no_outliers['Pass/Fail']

# 02 - Analyse de composantes principale (ACP)

# Standardisation des données
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Application de l'ACP
pca = PCA()
pca.fit(X_scaled)

# Variance expliquées par chaque composantes
explained_variance = pca.explained_variance_ratio_

# Cumul de la variance expliquée
cumulative_variance = np.cumsum(explained_variance)

# Choix du nombre de composantes pour conserver 95% de la variance expliquée
threshold = 0.95
n_components = np.argmax(cumulative_variance >= threshold) + 1
print(f"Nombre optimal de composantes pour {threshold*100}% de variance expliquée : {n_components}")

# Application de l'ACP avec le nombre de composantes jugées optimale
pca = PCA(n_components=n_components)
X_pca = pca.fit_transform(X_scaled)

# 03 - Tests de modèles d'IA

# Division des données en train/test
X_train, X_test, y_train, y_test = train_test_split(X_pca, Y, test_size=0.2, random_state=42)

# Tests de modèles

# Suréchantillonnage avec SMOTE et ADASYN
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train, y_train)

# Modèles et hyperparamètres
models = {
    'DecisionTreeClassifier': GridSearchCV(DecisionTreeClassifier(), {'max_depth': [3, 5, 10, None]}, cv=5),
    'RandomForestClassifier': GridSearchCV(RandomForestClassifier(), {'n_estimators': [50, 100, 200, 300], 'max_depth': [5, 10, None]}, cv=5),
    'AdaBoostClassifier': GridSearchCV(AdaBoostClassifier(), {'n_estimators': [50, 100, 200]}, cv=5),
    'MLPClassifier': GridSearchCV(MLPClassifier(max_iter=1000), {'hidden_layer_sizes': [(50,), (100,), (50, 50)], 'alpha': [0.0001, 0.001, 0.01]}, cv=5)
}

# Évaluation et journalisation avec WandB
def evaluate_models(X_train, y_train, label, run_id):
    results = {}
    plt.figure(figsize=(10, 6))
    for name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        results[name] = accuracy

        # Sauvegarde du modèle
        model_filename = f"{name}_{label}_run{run_id}.pkl"
        joblib.dump(model, model_filename)
        wandb.save(model_filename)

        # Courbe ROC
        fpr, tpr, _ = roc_curve(y_test, y_pred)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.2f})')

        # Journalisation WandB
        wandb.log({f"{name}_{label}_accuracy": accuracy, f"{name}_{label}_AUC": roc_auc})

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('Taux de faux positifs (FPR)')
    plt.ylabel('Taux de vrais positifs (TPR)')
    plt.title(f'Courbes ROC - {label}')
    plt.legend()
    plt.show()

    # Sauvegarde dans WandB
    wandb.log({f"ROC_{label}": wandb.Image(plt)})

    return results

# Lancement de 5 expériences avec variations aléatoires
total_runs = 5
for run in range(total_runs):
    wandb.init(
        project="suivi-avec-wandb",
        name=f"experiment_{run}",
        config={
            "learning_rate": round(random.uniform(0.001, 0.1), 3),
            "pca_components": 20,
            "dataset": "uci-secom",
            "epochs": 10
        }
    )

    performance_original = evaluate_models(X_train, y_train, 'Original', run)
    performance_smote = evaluate_models(X_train_smote, y_train_smote, 'SMOTE', run)
    performance_adasyn = evaluate_models(X_train_adasyn, y_train_adasyn, 'ADASYN', run)

    wandb.finish()